In [1]:
from platform import python_version
print(python_version())

3.6.5


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

%matplotlib inline

# Import data

In [3]:
# import datasets

forecast1 = pd.read_csv("Data/windforecasts_wf1.csv")
forecast2 = pd.read_csv("Data/windforecasts_wf2.csv")
forecast3 = pd.read_csv("Data/windforecasts_wf3.csv")
forecast4 = pd.read_csv("Data/windforecasts_wf4.csv")
forecast5 = pd.read_csv("Data/windforecasts_wf5.csv")
forecast6 = pd.read_csv("Data/windforecasts_wf6.csv")
forecast7 = pd.read_csv("Data/windforecasts_wf7.csv")
train = pd.read_csv("Data/train.csv")
benchmark = pd.read_csv("Data/benchmark.csv")

forecasts = [forecast1, forecast2, forecast3, forecast4, forecast5, forecast6, forecast7]
# Forecast structure: date hors u v ws wd
#train structure: date wp1 wp2 wp3 wp4 wp5 wp6 wp7

In [4]:
plot_titles = []
for i in range(7):
    plot_titles.append(f'Windfarm {i+1}')

plot_titles

['Windfarm 1',
 'Windfarm 2',
 'Windfarm 3',
 'Windfarm 4',
 'Windfarm 5',
 'Windfarm 6',
 'Windfarm 7']

# Functions

In [5]:
def plot_power_vs_direction(name, dir_series, wp_series):
    plt.scatter(dir_series, wp_series)
    plt.xlabel('Wind direction')
    plt.ylabel('Power production')
    plt.title(name)

In [6]:
# Function to remove overlapping forecasts from a forecast stored in a panda dataframe:

def remove_overlap(forecast_df):
    array = np.array([])
    r, c = forecast_df.shape
    for k in range(forecast_df.shape[1]):
        for i in range(0, forecast_df.shape[0], 48):
            for j in range(12):
                array = np.append(array, forecast_df.iloc[i+j, k])
    array2 = np.resize(array, (c, int(array.shape[0]/c)))
    fdict = {'Date': array2[0].astype(int), 'Hour': array2[1], 'u': array2[2], 'v': array2[3], 'wind_speed': array2[4], 'wind_dir': array2[5]}
    df = pd.DataFrame(data=fdict)
    return df

In [7]:
def convert_date_format(forecast_dataframe):
    for i in range (forecast_dataframe.shape[0]):
        # Fetch the date in its original format:
        prev_date = str(forecast_dataframe.iloc[i,0])
        forecast_hour = int(prev_date[-2:])
        
        # Fetch the hour the forecast is for:
        # FIX THIS: 1-12 INSTEAD OF 0-23
        hours_ahead = int(forecast_dataframe.iloc[i,1])
        hour = str(forecast_hour + hours_ahead).zfill(2)
        
        # Strip the hours the forecast was made
        date = prev_date[:-2]
        
        #append the hour the forecast is for at the end of the date:
        date_time = date + hour
        
        # Replace the date with the new date format:
        forecast_dataframe.iloc[i,0] = date_time
    
    return forecast_dataframe

# Cleaning
## Define functions:

In [9]:
# Function to remove overlapping forecasts from a forecast stored in a panda dataframe:

def remove_overlap(forecast_df):
    array = np.array([])
    r, c = forecast_df.shape
    for k in range(forecast_df.shape[1]):
        for i in range(0, forecast_df.shape[0], 48):
            for j in range(12):
                array = np.append(array, forecast_df.iloc[i+j, k])
    array2 = np.resize(array, (c, int(array.shape[0]/c)))
    fdict = {'date': array2[0].astype(int), 'hors': array2[1], 'ws': array2[4], 'wd': array2[5]}
    df = pd.DataFrame(data=fdict)
    return df

# datestring format: yyyymmddHH
def convert_to_datetime(datestring):
    day_offset = dt.timedelta(hours=24)
    # Catch change of day and give proper datetime:
    if (datestring[-2:] == '24'):
        basedate = dt.datetime.strptime(datestring[:-2], '%Y%m%d')
        adjusted_datetime_string = dt.datetime.strftime(basedate, '%Y%m%d') + '00'
        new_date = dt.datetime.strptime(adjusted_datetime_string, '%Y%m%d%H') + day_offset
    else:
        new_date = dt.datetime.strptime(datestring, '%Y%m%d%H')
    
    return new_date

def convert_date_format(forecast_dataframe):
    for i in range (forecast_dataframe.shape[0]):
        # Fetch the date in its original format:
        prev_date = str(forecast_dataframe.iloc[i,0])
        forecast_hour = int(prev_date[-2:])
        
        # Fetch the hour the forecast is for:
        hours_ahead = int(forecast_dataframe.iloc[i,1])
        hour = str(forecast_hour + hours_ahead).zfill(2)
        
        # Replace the hours the forecast was made with the forecasted hour
        date = prev_date[:-2] + hour
        
        #append the hour the forecast is for at the end of the date:
        #date_time = date + hour
        
        forecast_dataframe.iloc[i,0] = date
    
    # Convert from string to datetime:
    forecast_dataframe['date'] = forecast_dataframe['date'].apply(convert_to_datetime)
    
    # Use datetime as index:
    forecast_dataframe.set_index('date', drop=False, inplace=True, verify_integrity=True)
    
    return forecast_dataframe

def prepare_columns(forecast_df):
    forecast_df.drop(['hors'], axis=1, inplace=True)
    
    forecast_df.insert(1, 'ws-2', 0.0)
    #Date0 ws-2[1] wind_speed[2] wind_dir[3]
    forecast_df.insert(2, 'ws-1', 0.0)
    forecast_df.insert(4, 'ws+1', 0.0)
    #Date[0] ws-2[1] ws-1[2] wind_speed[3] ws+1[4] wind_dir[5]
    forecast_df.insert(5, 'wd-2', 0.0)
    forecast_df.insert(6, 'wd-1', 0.0)
    forecast_df.insert(8, 'wd+1', 0.0)

    features = ['hour_from_06', 'week', 'month']
    for feature in features:
        forecast_df.insert(forecast_df.shape[1], feature, 0)
        
    forecast_df.insert(forecast_df.shape[1], 'production', 0.0)
    
    return forecast_df

def create_features(df_in):
    # Create output dataframe excluding two first rows and last (for ws-2 .. ws+1 features)
    df_out = df_in.iloc[2:-1,:]
    
    # Create timedeltas:
    d2 = dt.timedelta(hours=-2)
    d1 = dt.timedelta(hours=-1)
    dp1 = dt.timedelta(hours=1)

    for i in df_out['date']:
        # Creating windspeed features:
        df_out.loc[i, 'ws-2'] = df_in.loc[i+d2, 'ws']
        df_out.loc[i, 'ws-1'] = df_in.loc[i+d1, 'ws']
        df_out.loc[i, 'ws+1'] = df_in.loc[i+dp1, 'ws']

        # Creating wind direction features:
        df_out.loc[i, 'wd-2'] = df_in.loc[i+d2, 'wd']
        df_out.loc[i, 'wd-1'] = df_in.loc[i+d1, 'wd']
        df_out.loc[i, 'wd+1'] = df_in.loc[i+dp1, 'wd']
        
        # Setting week number:
        df_out.loc[i, 'week'] = df_out.loc[i, 'date'].isocalendar()[1]
        
        #setting month number:
        df_out.loc[i, 'month'] = df_out.loc[i, 'date'].month
        
        # setting time from 06:00:
        df_out.loc[i, 'hour_from_06'] = abs(df_out.loc[i, 'date'].hour - 6)
        
        
    return df_out

# create features only for the dates we have production
def create_features_v2(df_in, train_df):
    # Create output dataframe excluding two first rows and last (for ws-2 .. ws+1 features)
    df_out = df_in.iloc[2:-1,:]
    df_out = df_out.loc[df_out['date'].isin(train_df['date'])]
    
    # Create timedeltas:
    d2 = dt.timedelta(hours=-2)
    d1 = dt.timedelta(hours=-1)
    dp1 = dt.timedelta(hours=1)
    counthour = 1
    
    for i in df_out['date']:
        # Creating windspeed features:
        df_out.loc[i, 'ws-2'] = df_in.loc[i+d2, 'ws']
        df_out.loc[i, 'ws-1'] = df_in.loc[i+d1, 'ws']
        df_out.loc[i, 'ws+1'] = df_in.loc[i+dp1, 'ws']

        # Creating wind direction features:
        df_out.loc[i, 'wd-2'] = df_in.loc[i+d2, 'wd']
        df_out.loc[i, 'wd-1'] = df_in.loc[i+d1, 'wd']
        df_out.loc[i, 'wd+1'] = df_in.loc[i+dp1, 'wd']
        
        # Setting week number:
        df_out.loc[i, 'week'] = df_out.loc[i, 'date'].isocalendar()[1]
        
        #setting month number:
        df_out.loc[i, 'month'] = df_out.loc[i, 'date'].month
        
        # setting time from 06:00:
        df_out.loc[i, 'hour_from_06'] = abs(df_out.loc[i, 'date'].hour)
        df_out.loc[i, 'hour_from_06'] = hoursFrom06(df_out.loc[i, 'hour_from_06'])
        
        
    return df_out


def prepare_train_data(train_df):
    train_df['date'] = train['date'].apply(str)
    train_df['date'] = train['date'].apply(convert_to_datetime)
    train_df.set_index('date', drop=False, inplace=True, verify_integrity=True)
    
    return train_df

def hoursFrom06(time):
    baseline = 6
    if time > 12:
        return 12 - abs((time - baseline)-12)
    return abs(time - baseline)

In [10]:
tdf = remove_overlap(forecast1)
tdf.head()

,date,hors,ws,wd
0,2009070100,1.0,2.47,108.68
1,2009070100,2.0,2.40,114.31
2,2009070100,3.0,2.51,118.71
3,2009070100,4.0,2.73,120.86
4,2009070100,5.0,2.93,120.13


In [13]:
test = tdf
test.head()

,date,hors,ws,wd
0,2009070100,1.0,2.47,108.68
1,2009070100,2.0,2.40,114.31
2,2009070100,3.0,2.51,118.71
3,2009070100,4.0,2.73,120.86
4,2009070100,5.0,2.93,120.13


In [14]:
test = convert_date_format(test)
test.head()

,date,hors,ws,wd
date,,,,
2009-07-01 01:00:00,2009-07-01 01:00:00,1.0,2.47,108.68
2009-07-01 02:00:00,2009-07-01 02:00:00,2.0,2.40,114.31
2009-07-01 03:00:00,2009-07-01 03:00:00,3.0,2.51,118.71
2009-07-01 04:00:00,2009-07-01 04:00:00,4.0,2.73,120.86
2009-07-01 05:00:00,2009-07-01 05:00:00,5.0,2.93,120.13


In [15]:
def prepare_train_data(train_df):
    train_df['date'] = train['date'].apply(str)
    train_df['date'] = train['date'].apply(convert_to_datetime)
    train_df.set_index('date', drop=False, inplace=True, verify_integrity=True)
    
    return train_df

In [16]:
train['date'] = train['date'].apply(str)
train['date'] = train['date'].apply(convert_to_datetime)
train.head()

,date,wp1,wp2,wp3,wp4,wp5,wp6,wp7
0,2009-07-01 00:00:00,0.045,0.233,0.494,0.105,0.056,0.118,0.051
1,2009-07-01 01:00:00,0.085,0.249,0.257,0.105,0.066,0.066,0.051
2,2009-07-01 02:00:00,0.020,0.175,0.178,0.033,0.015,0.026,0.000
3,2009-07-01 03:00:00,0.060,0.085,0.109,0.022,0.010,0.013,0.000
4,2009-07-01 04:00:00,0.045,0.032,0.079,0.039,0.010,0.000,0.000


In [17]:
train.set_index('date', drop=False, inplace=True, verify_integrity=True)
train.head()

,date,wp1,wp2,wp3,wp4,wp5,wp6,wp7
date,,,,,,,,
2009-07-01 00:00:00,2009-07-01 00:00:00,0.045,0.233,0.494,0.105,0.056,0.118,0.051
2009-07-01 01:00:00,2009-07-01 01:00:00,0.085,0.249,0.257,0.105,0.066,0.066,0.051
2009-07-01 02:00:00,2009-07-01 02:00:00,0.020,0.175,0.178,0.033,0.015,0.026,0.000
2009-07-01 03:00:00,2009-07-01 03:00:00,0.060,0.085,0.109,0.022,0.010,0.013,0.000
2009-07-01 04:00:00,2009-07-01 04:00:00,0.045,0.032,0.079,0.039,0.010,0.000,0.000


In [15]:
train['date'].head()

date
2009-07-01 00:00:00   2009-07-01 00:00:00
2009-07-01 01:00:00   2009-07-01 01:00:00
2009-07-01 02:00:00   2009-07-01 02:00:00
2009-07-01 03:00:00   2009-07-01 03:00:00
2009-07-01 04:00:00   2009-07-01 04:00:00
Name: date, dtype: datetime64[ns]

In [18]:
test = prepare_columns(test)

In [19]:
test.head()

,date,ws-2,ws-1,ws,ws+1,wd-2,wd-1,wd,wd+1,hour_from_06,week,month,production
date,,,,,,,,,,,,,
2009-07-01 01:00:00,2009-07-01 01:00:00,0.0,0.0,2.47,0.0,0.0,0.0,108.68,0.0,0,0,0,0.0
2009-07-01 02:00:00,2009-07-01 02:00:00,0.0,0.0,2.40,0.0,0.0,0.0,114.31,0.0,0,0,0,0.0
2009-07-01 03:00:00,2009-07-01 03:00:00,0.0,0.0,2.51,0.0,0.0,0.0,118.71,0.0,0,0,0,0.0
2009-07-01 04:00:00,2009-07-01 04:00:00,0.0,0.0,2.73,0.0,0.0,0.0,120.86,0.0,0,0,0,0.0
2009-07-01 05:00:00,2009-07-01 05:00:00,0.0,0.0,2.93,0.0,0.0,0.0,120.13,0.0,0,0,0,0.0


In [21]:
test1 = create_features_v2(test, train)


In [22]:
test1[:][1:24]

,date,ws-2,ws-1,ws,ws+1,wd-2,wd-1,wd,wd+1,hour_from_06,week,month,production
date,,,,,,,,,,,,,
2009-07-01 04:00:00,2009-07-01 04:00:00,2.40,2.51,2.73,2.93,114.31,118.71,120.86,120.13,2,27,7,0.0
2009-07-01 05:00:00,2009-07-01 05:00:00,2.51,2.73,2.93,2.96,118.71,120.86,120.13,115.79,1,27,7,0.0
2009-07-01 06:00:00,2009-07-01 06:00:00,2.73,2.93,2.96,2.81,120.86,120.13,115.79,106.71,0,27,7,0.0
2009-07-01 07:00:00,2009-07-01 07:00:00,2.93,2.96,2.81,2.73,120.13,115.79,106.71,95.39,1,27,7,0.0
2009-07-01 08:00:00,2009-07-01 08:00:00,2.96,2.81,2.73,2.87,115.79,106.71,95.39,88.50,2,27,7,0.0
2009-07-01 09:00:00,2009-07-01 09:00:00,2.81,2.73,2.87,3.23,106.71,95.39,88.50,90.19,3,27,7,0.0
2009-07-01 10:00:00,2009-07-01 10:00:00,2.73,2.87,3.23,3.66,95.39,88.50,90.19,95.15,4,27,7,0.0
2009-07-01 11:00:00,2009-07-01 11:00:00,2.87,3.23,3.66,3.94,88.50,90.19,95.15,98.71,5,27,7,0.0
2009-07-01 12:00:00,2009-07-01 12:00:00,3.23,3.66,3.94,2.85,90.19,95.15,98.71,103.17,6,27,7,0.0


In [23]:
test1.head()
len(test1)

18754

In [24]:
for i in test1['date']:
    test1.loc[i, 'production'] = train.loc[i, 'wp1']
    
test1.head()

,date,ws-2,ws-1,ws,ws+1,wd-2,wd-1,wd,wd+1,hour_from_06,week,month,production
date,,,,,,,,,,,,,
2009-07-01 03:00:00,2009-07-01 03:00:00,2.47,2.40,2.51,2.73,108.68,114.31,118.71,120.86,3,27,7,0.060
2009-07-01 04:00:00,2009-07-01 04:00:00,2.40,2.51,2.73,2.93,114.31,118.71,120.86,120.13,2,27,7,0.045
2009-07-01 05:00:00,2009-07-01 05:00:00,2.51,2.73,2.93,2.96,118.71,120.86,120.13,115.79,1,27,7,0.035
2009-07-01 06:00:00,2009-07-01 06:00:00,2.73,2.93,2.96,2.81,120.86,120.13,115.79,106.71,0,27,7,0.005
2009-07-01 07:00:00,2009-07-01 07:00:00,2.93,2.96,2.81,2.73,120.13,115.79,106.71,95.39,1,27,7,0.000


In [22]:
test1.drop('date', axis=1, inplace=True)
test1.head()

,ws-2,ws-1,ws,ws+1,wd-2,wd-1,wd,wd+1,hour_from_06,week,month,production
date,,,,,,,,,,,,
2009-07-01 03:00:00,2.47,2.40,2.51,2.73,108.68,114.31,118.71,120.86,3,27,7,0.060
2009-07-01 04:00:00,2.40,2.51,2.73,2.93,114.31,118.71,120.86,120.13,2,27,7,0.045
2009-07-01 05:00:00,2.51,2.73,2.93,2.96,118.71,120.86,120.13,115.79,1,27,7,0.035
2009-07-01 06:00:00,2.73,2.93,2.96,2.81,120.86,120.13,115.79,106.71,0,27,7,0.005
2009-07-01 07:00:00,2.93,2.96,2.81,2.73,120.13,115.79,106.71,95.39,1,27,7,0.000


In [25]:
test1.to_csv('sample_wp1_full.csv', index=False)

In [24]:
train1 = prepare_train_data(train)
train1.head()

ValueError: time data '2009-07-01 00:00:00' does not match format '%Y%m%d%H'